In [2]:
from pandas.errors import InvalidIndexError
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
%matplotlib inline

df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


In [45]:
df.replace('Договорная',52000,inplace=True)
df.replace('Площадьземли:1сот',50,inplace=True)

In [ ]:
df['district'].value_counts()

In [48]:
df['size']=pd.to_numeric(df['size'])
df['price']=pd.to_numeric(df['price'])

In [108]:
from sklearn.model_selection import train_test_split
train_set,test_set=train_test_split(df, test_size=0.2, random_state=42)
X_train=train_set.drop('price',axis=1)
y_train=train_set['price'].copy()
df_test=train_set.drop(['location','district','price'],axis=1)
df_columns=train_set.drop(['location','district','price'],axis=1)
df_cat=train_set['district'].copy()

In [109]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [131]:
def std(dataset):
  Std_scaler=StandardScaler()
  return Std_scaler.fit_transform(dataset)
def encoder(cat_dataset):
  cat_attribs=['district']
  pipeline=ColumnTransformer([
                              ('cat',OrdinalEncoder(),cat_attribs)
  ])
  return pipeline.fit_transform(cat_dataset)
df_num=std(df_test)
district=encoder(X_train)
df_ready=pd.DataFrame(df_num,columns=df_columns.columns, index=X_train.index)
df_labels=pd.DataFrame(district,columns=['district'],index=X_train.index)
df_ready['district']=df_labels['district']

,rooms,size,level,max_levels,district
4552,-0.577461,-0.038099,-0.312669,-0.789505,5.0
4450,1.251317,0.067940,-0.758644,-0.403304,2.0
1656,-1.491850,-0.048311,-0.758644,-0.403304,9.0
3570,-0.577461,-0.020819,-0.312669,-0.017102,8.0
6618,1.251317,0.050660,0.133306,0.369100,7.0
...,...,...,...,...,...
5191,0.336928,-0.023961,0.133306,-0.789505,8.0
5226,1.251317,-0.006680,2.363183,1.141503,6.0
5390,-0.577461,-0.040456,-1.204620,-1.175707,6.0
860,-0.577461,-0.031816,-0.312669,-0.789505,7.0


In [138]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(df_ready, y_train)

RandomForestRegressor()

In [139]:
predict=RF_model.predict(df_ready)
predict

array([ 38097.0952381 , 213659.        ,  26802.20165749, ...,
        52354.74603175,  49495.34126984,  57543.33333333])

In [140]:
pd.DataFrame({'Prognoz':predict, 'Real baxosi': y_train})

,Prognoz,Real baxosi
4552,38097.095238,37000
4450,213659.000000,210000
1656,26802.201657,27500
3570,48039.746667,46835
6618,131540.000000,140000
...,...,...
5191,54874.291382,43500
5226,58648.183333,56000
5390,52354.746032,58500
860,49495.341270,53000


In [141]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(predict,y_train)
xatolik=np.sqrt(mse)
print(xatolik)

339741.2929049208


In [144]:
import pickle 
filename='RF_model.pkl'
with open(filename,'wb') as file:
  pickle.dump(RF_model,file)